# Using LLAMA2 via API

An application programming interface (API) facilitates communication between computer programs, serving as a software interface enabling services to be shared between them. Unlike a user interface, which connects a computer to a person, an API links computers or software components together, typically accessed by programmers rather than end users for software integration.

More information: https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Prompt_Engineering_with_Llama_2.ipynb

In [3]:
!pip install langchain replicate

     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     -------------------------------------- 103.4/103.4 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
     ---------------------------------------- 50.9/50.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/817.7 kB ? eta -:--:--
   ------------------- -------------------- 399.4/817.7 kB 8.5 MB/s eta 0:00:01
   --------------------------------------  798.7/817.7 kB 10.1 MB/s eta 0:00:01
   --------------------------------------  798.7/817.7 kB 10.1 MB/s eta 0:00:01
   --------------------------------------  798.7/817.7 kB 10.1 MB/s eta 0:00:01
   --------------------------------------  798.7/817.7 kB 10.1 MB/s eta 0:00:01
   --------------------------------------  798.7/817.7 kB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 817.7/817.7 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/371.6 kB ? eta -:--:--


Get a free API key from
https://replicate.com/account/api-tokens

In [4]:
import os

# Define the path to the text file, assuming it's two folders up from the current directory
text_file_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'replicate key.txt'))

# Open the text file
try:
    with open(text_file_path, 'r') as file:
        # Do something with the file, for example, read its contents
        replicate_key = file.read()
        print(file_contents)
except FileNotFoundError:
    print("The text file does not exist in the specified location.")
except Exception as e:
    print("An error occurred:", e)


r8_6qozFGfh5gVlzuAnfrAEsG16QvxVuKI1yqlpS


In [5]:
YOUR_KEY_HERE = replicate_key

In [6]:
from typing import Dict, List
from langchain.llms import Replicate
from langchain.memory import ChatMessageHistory
from langchain.schema.messages import get_buffer_string
import os

os.environ["REPLICATE_API_TOKEN"] = YOUR_KEY_HERE

In [7]:
LLAMA2_13B_CHAT = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

Other models can be found here: https://replicate.com/meta

# Parameters

**prompt** – (required) the prompt that you want to pass to the model.

**temperature**  controls randomness. Higher temperature makes outputs more random and diverse. Lower temperature produces more stable and repetitive completions. A temperature of zero effectively disables random sampling and makes the completions deterministic.

**top-p** is the cumulative probability cutoff for token selection, i.e. the percentile of probability from which tokens are sampled. Lower values mean sampling from a smaller set of more top-weighted tokens. Lower top-p values reduce diversity and focus on more probable tokens. A value lower than 1.0 means that only the corresponding top percentile of options are considered, and that less likely options will not be generated, resulting in more stable and repetitive completions.

**top-k** samples from the k most likely next tokens at each step. Lower k focuses on higher probability tokens.

**max_new_tokens** specifies the maximum number of tokens to use in the generated response. The model truncates the response once the generated text exceeds max_new_tokens.

#Roles
**System**'s  messages are used to provide core instruction to the LLM.

**User**'s messages are human-provided messages.

**Assistant**'s messages are generated by the LLM.

In [12]:
DEFAULT_MODEL = LLAMA2_13B_CHAT

def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    llm = Replicate(
        model=model,
        model_kwargs={"temperature": temperature,"top_p": top_p, "max_new_tokens": 1000}
    )
    return llm(prompt)

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    history = ChatMessageHistory()
    for message in messages:
        if message["role"] == "user":
            history.add_user_message(message["content"])
        elif message["role"] == "assistant":
            history.add_ai_message(message["content"])
        else:
            raise Exception("Unknown role")
    return completion(
        get_buffer_string(
            history.messages,
            human_prefix="USER",
            ai_prefix="ASSISTANT",
        ),
        model,
        temperature,
        top_p,
    )

def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    print(f'==============\n{prompt}\n==============')
    response = completion(prompt, model)
    print(response, end='\n\n')

In [11]:
complete_and_print("What is the transformer AI architecture?")

What is the transformer AI architecture?
 Hello! I'd be happy to help you understand the Transformer AI architecture.

The Transformer architecture is a type of neural network designed primarily for natural language processing tasks, such as machine translation, question answering, and text classification. It was introduced in a paper by Vaswani et al. in 2017 and has since become one of the most widely used architectures in the field.

The key innovation of the Transformer is the self-attention mechanism, which allows the model to attend to different parts of the input sequence simultaneously and weigh their importance. This is in contrast to traditional recurrent neural networks (RNNs), which process the input sequence sequentially and have recurrence connections that allow them to capture long-range dependencies.

The Transformer architecture consists of an encoder and a decoder. The encoder takes in a sequence of words or tokens and outputs a continuous representation of the input 

In [ ]:
complete_and_print("What is the transformer AI architecture? Please answer in one sentence.")

What is the transformer AI architecture? Please answer in one sentence.
 The Transformer architecture is a type of neural network designed for natural language processing tasks, characterized by self-attention mechanisms and parallelization across input sequences, allowing it to efficiently process long input sequences and handle complex contextual relationships.



In [ ]:
complete_and_print("What is API in computer science? Please answer in one sentence.")

What is API in computer science? Please answer in one sentence.
 Sure, I'd be happy to help! In computer science, an Application Programming Interface (API) is a set of rules and protocols that allows different software systems to communicate with each other and exchange data, much like how a messenger delivers messages between two parties.



In [ ]:
response = chat_completion(messages=[
    user("UBO is a French university, located in Brest, in the Academy of Rennes. "),
    assistant("That's great to hear!"),
    user("What is the location of UBO?"),
])
print(response)

 ASSISTANT: Ah, you're looking for information about the University of Brittany Occidental (UBO), located in Brest, which is part of the Academy of Rennes. Well, let me check my maps... (checks map) Oh, I see! UBO is situated in the beautiful city of Brest, which is located in the western part of France, specifically in the region of Brittany. It's a lovely coastal city with a rich history and culture, known for its picturesque harbor and delicious seafood. The university is easily accessible by car or train, and it's just a short walk from the city center. Is there anything else you'd like to know about UBO or Brest?


# Different prompting techniques

These prompting techniques are valid for any LLM-bsed chat (ChatGPT, LLAMA Chat, Jurrasic-2 Chat, Gemini,...).

For more information see: https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Prompt_Engineering_with_Llama_2.ipynb


## 1. Explicit Instructions


*   Stylization (Explain Like I'm Five)
*   Format (json, bulleted list)
*   Restrictions (only ...)






## 2. Zero-Shot Prompting
Generate answer without giving examples



In [ ]:
complete_and_print("Text: Old professors never die they just loose their universities. \n Does this texts contain a pun?")

Text: Old professors never die they just loose their universities. 
 Does this texts contain a pun?
 Ah, you've provided me with a clever text that indeed contains a pun! "Old professors never die, they just lose their universities" is a playful take on the phrase "old soldiers never die," but repurposed to fit the context of aging academics and their institutions. The wordplay lies in the double meaning of "lose": both as a verb (to misplace or be defeated) and as a noun (a university). Well done, my young apprentice!



## 3. Few-Shot Prompting

In [ ]:
def is_pun(text):
    response = chat_completion(messages=[
        user("You are a joke classifier. For each message, indicate if it contains a pun."),
        user("She was suspected of stealing a brooch but they couldn't pin it on her."),
        assistant("Pun"),
        user("An apple a day keeps the doctor away"),
        assistant("Not pun"),
        user("The inept mathematician couldn't count on his friends."),
        assistant("Pun"),
        user(text),
    ])
    return response

def print_is_pun(text):
    print(f'INPUT: {text}')
    print(is_pun(text))

print_is_pun("Old professors never die they just loose their universities.")

INPUT: Old professors never die they just loose their universities.
 ASSISTANT: Pun


## 4. Role Prompting

In [ ]:
complete_and_print("Your role is a professor who explains highly technical concepts in a very difficult way. Explain what is LLM. Please be less quirky.")

Your role is a professor who explains highly technical concepts in a very difficult way. Explain what is LLM. Please be less quirky.
 Greetings, my dear student! Today, we shall embark on a fascinating journey through the realm of machine learning, specifically discussing the concept of Language Model (LLM). Prepare thyself for an intellectual odyssey that may challenge thy cognitive abilities, but fear not, for I shall guide thee with patience and clarity.

LLM, or Language Model, refers to a computational framework designed to process and analyze human language. This encompasses various aspects, such as grammar, syntax, semantics, and pragmatics. The primary objective of LLM is to enable machines to comprehend and generate human-like language, thereby facilitating seamless communication between humans and machines.

To achieve this goal, LLM employs sophisticated algorithms and techniques from natural language processing (NLP), including deep learning methods like recurrent neural ne

## 5. Chain-of-Thought


1.   Break Down Your Question
2.   Be Specific, Include Context
3.   Give feedback

### Different Ways to Use Chain of Thought Prompting


1.   Start general, then dive into specifics
2.   Comparative Prompting (contrast things)
3. Thought Process Prompting
  *   "..., think step by step"
  *   "..., thinking aloud"
  *   "..., explain your thoughts"
4.  Ask for a sequence or progression





# Applying LLM to a Pandas DataFrame
Source: https://drive.google.com/file/d/1PMmHCXnU5pSO0v1stMwjipwbD7TswOrr/view?usp=drive_link



In [13]:
import pandas as pd

In [22]:
train=pd.read_json('../../various data/simpletext_task3_train.json')
train


,query_id,query_text,doc_id,snt_id,source_snt
0,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ..."
1,G11.1,drones,2892036907,G11.1_2892036907_2,With the ever increasing number of unmanned ae...
2,G11.1,drones,2892036907,G11.1_2892036907_3,Due to guidelines set by the governments regar...
3,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...
4,G11.1,drones,2892036907,G11.1_2892036907_5,Derived from the classic image classification ...
...,...,...,...,...,...
643,M9,Mechanisms of Muscle Hypertrophy,41,M9_41_3,Bodybuilders generally train with moderate loa...
644,M9,Mechanisms of Muscle Hypertrophy,41,M9_41_4,"Powerlifters, on the other hand, routinely tra..."
645,M9,Mechanisms of Muscle Hypertrophy,41,M9_41_5,Although both groups are known to display impr...
646,M9,Mechanisms of Muscle Hypertrophy,41,M9_41_6,It has been shown that many factors mediate th...


In [23]:
prompt_simplification="""
    You are a robot that only outputs JSON.
    You reply in JSON format with the field 'simplification'.
    You provide a simplification of a source sentence only.
    You do not add 'Sure! Here\'s the simplification of your sentence:' nor additional spaces nor new lines.
    Example source sentence: In the modern era of automation and robotics, \
    autonomous vehicles are currently the focus of academic and industrial research.? \
    Example answer: {'simplification': 'Current academic and industrial research is interested in autonomous vehicles.'}
    Now here is my sentence:
"""

In [27]:
c=completion(prompt_simplification+train['source_snt'][0])
print(prompt_simplification+train['source_snt'][0])
c


    You are a robot that only outputs JSON.
    You reply in JSON format with the field 'simplification'.
    You provide a simplification of a source sentence only.
    You do not add 'Sure! Here's the simplification of your sentence:' nor additional spaces nor new lines.
    Example source sentence: In the modern era of automation and robotics,     autonomous vehicles are currently the focus of academic and industrial research.?     Example answer: {'simplification': 'Current academic and industrial research is interested in autonomous vehicles.'}
    Now here is my sentence:
In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research.



' { "simplification": "Research is focused on autonomous vehicles." }'

In [25]:
import json
tt=json.loads(completion(prompt_simplification+train['source_snt'][0]))
tt

{'simplification': 'Research is focused on autonomous vehicles.'}

In [ ]:
tt['simplification']

'Research is focused on autonomous vehicles.'

In [ ]:
json.loads(completion(prompt_simplification+train['source_snt'][1]))['simplification']

'There is a growing need for autonomy in unmanned aerial vehicles (UAVs) as they become more prevalent in civilian and commercial activities.'

In [28]:
train=train.head(5)

In [29]:
train

,query_id,query_text,doc_id,snt_id,source_snt
0,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ..."
1,G11.1,drones,2892036907,G11.1_2892036907_2,With the ever increasing number of unmanned ae...
2,G11.1,drones,2892036907,G11.1_2892036907_3,Due to guidelines set by the governments regar...
3,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...
4,G11.1,drones,2892036907,G11.1_2892036907_5,Derived from the classic image classification ...


In [30]:
train['simplification']=train["source_snt"].apply(lambda x:json.loads(completion(prompt_simplification+x))['simplification'])
train

C:\Users\Master\AppData\Local\Temp\ipykernel_18884\3892787995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['simplification']=train["source_snt"].apply(lambda x:json.loads(completion(prompt_simplification+x))['simplification'])


,query_id,query_text,doc_id,snt_id,source_snt,simplification
0,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ...",Research is focused on autonomous vehicles.
1,G11.1,drones,2892036907,G11.1_2892036907_2,With the ever increasing number of unmanned ae...,There is a growing need for autonomy in unmann...
2,G11.1,drones,2892036907,G11.1_2892036907_3,Due to guidelines set by the governments regar...,Government regulations are driving interest in...
3,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...,We propose a data-driven solution for UAV auto...
4,G11.1,drones,2892036907,G11.1_2892036907_5,Derived from the classic image classification ...,Our image classification algorithm uses a 39-l...


In [ ]:
prompt_terms="""
    You are a robot that ONLY outputs JSON.
    You reply in JSON format with the field 'terms'.
    You provide semicolon-separated  list of scientific terms of a source sentence ONLY.
    You DO NOT add 'Sure! Here are the scientific terms of your sentence:'.
    Example source sentence: In the modern era of automation and robotics, \
    autonomous vehicles are currently the focus of academic and industrial research.? \
    Example answer: {'terms': 'robotics; autonomous vehicles'}
    Now here is my sentence:
"""

In [ ]:
c=completion(prompt_terms+train['source_snt'][0])
c

' {\n"terms": "autonomous vehicles; robotics; automation"\n}'

In [ ]:
json.loads(c)

{'terms': 'autonomous vehicles; robotics; automation'}

In [ ]:
train['all_terms']=train["source_snt"].apply(lambda x:json.loads(completion(prompt_terms+x))['terms'])
train

<ipython-input-30-3e19c191d485>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['all_terms']=train["source_snt"].apply(lambda x:json.loads(completion(prompt_terms+x))['terms'])


,query_id,query_text,doc_id,snt_id,source_snt,simplification,all_terms
0,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ...",Research is focused on autonomous vehicles.,automation; robotics; autonomous vehicles
1,G11.1,drones,2892036907,G11.1_2892036907_2,With the ever increasing number of unmanned ae...,There is a growing need for autonomy in unmann...,unmanned aerial vehicles; autonomy
2,G11.1,drones,2892036907,G11.1_2892036907_3,Due to guidelines set by the governments regar...,Government regulations are driving interest in...,civil drones; road-tracking based navigation
3,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...,We propose a data-driven solution for UAV auto...,UAV autonomy; expert pilot; data-driven soluti...
4,G11.1,drones,2892036907,G11.1_2892036907_5,Derived from the classic image classification ...,Our image classification algorithm uses a 39-l...,image classification; Inception model; tomogra...


In [ ]:
train['terms']=train['all_terms'].str.split(";")
train=train.explode('terms').reset_index(drop=True)
train.drop_duplicates(inplace=True,subset=['snt_id','terms'])
train

<ipython-input-31-73bdd7af7009>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['terms']=train['all_terms'].str.split(";")


,query_id,query_text,doc_id,snt_id,source_snt,simplification,all_terms,terms
0,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ...",Research is focused on autonomous vehicles.,automation; robotics; autonomous vehicles,automation
1,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ...",Research is focused on autonomous vehicles.,automation; robotics; autonomous vehicles,robotics
2,G11.1,drones,2892036907,G11.1_2892036907_1,"In the modern era of automation and robotics, ...",Research is focused on autonomous vehicles.,automation; robotics; autonomous vehicles,autonomous vehicles
3,G11.1,drones,2892036907,G11.1_2892036907_2,With the ever increasing number of unmanned ae...,There is a growing need for autonomy in unmann...,unmanned aerial vehicles; autonomy,unmanned aerial vehicles
4,G11.1,drones,2892036907,G11.1_2892036907_2,With the ever increasing number of unmanned ae...,There is a growing need for autonomy in unmann...,unmanned aerial vehicles; autonomy,autonomy
5,G11.1,drones,2892036907,G11.1_2892036907_3,Due to guidelines set by the governments regar...,Government regulations are driving interest in...,civil drones; road-tracking based navigation,civil drones
6,G11.1,drones,2892036907,G11.1_2892036907_3,Due to guidelines set by the governments regar...,Government regulations are driving interest in...,civil drones; road-tracking based navigation,road-tracking based navigation
7,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...,We propose a data-driven solution for UAV auto...,UAV autonomy; expert pilot; data-driven soluti...,UAV autonomy
8,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...,We propose a data-driven solution for UAV auto...,UAV autonomy; expert pilot; data-driven soluti...,expert pilot
9,G11.1,drones,2892036907,G11.1_2892036907_4,In an attempt to achieve the above mentioned t...,We propose a data-driven solution for UAV auto...,UAV autonomy; expert pilot; data-driven soluti...,data-driven solution


# Excercise (5-10 source sentences MAX)

1.   Experiment with different prompts for text simplification. For each prompt, create a new column with generated outputs. Add the ID (name) of the prompt to the column header. Keep track of your prompts.
2.   Find all difficult scientific terms in each source sentence. For each scientific term, provide its level of difficulty, definition, explanation, and examples in separate columns. Keep track of your prompts.
3. Add Wikipedia definitions based on the Introduction course (https://pypi.org/project/wikipedia/)
4. Predict if a given text contains a pun (5-10 source sentences MAX) https://drive.google.com/file/d/1kheFk2vMaIedQCfSFiF4VA45CU4_-xfv/view?usp=drive_link Keep track of your prompts
5. Translate the following puns into French (5-10 source sentences MAX) https://drive.google.com/file/d/1z_7ym2Bh2GmVqZW74FtYQaaY8dMEn-rJ/view?usp=drive_link Keep track of your prompts
6. Register at https://clef2024.clef-initiative.eu/index.php?page=Pages/registration.html Choose the SimpleText and JOKER labs.



